In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression

In [6]:
train_df = pd.read_csv("dataset/train.csv", header=None)
test_df = pd.read_csv("dataset/test.csv", header=None)

train_df.columns = ["rating", "review"]
test_df.columns = ["rating", "review"]

In [7]:
train_df.head()

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [10]:
test_df.head()

,rating,review
0,2,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,1,The food is good. Unfortunately the service is...
4,2,Even when we didn't have a car Filene's Baseme...


In [9]:
train_df["rating"].value_counts()

1    280000
2    280000
Name: rating, dtype: int64

In [11]:
test_df["rating"].value_counts()

2    19000
1    19000
Name: rating, dtype: int64

In [15]:
y = train_df.rating.values

In [24]:
X_train, X_valid, y_train, y_valid = train_test_split(
    train_df.review.values,
    y,
    stratify=y,
    random_state=42,
    test_size=0.2,
    shuffle=True
)

In [19]:
%%time
tfv = TfidfVectorizer(
    min_df=3,
    max_features=None, 
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 3),
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=True,
    stop_words='english'
)

tfv.fit(list(X_train) + list(X_valid))
xtrain_transform  = tfv.transform(X_train)
xvalid_transform  = tfv.transform(X_valid)

CPU times: user 2min 16s, sys: 2.01 s, total: 2min 18s
Wall time: 2min 21s


In [21]:
clf = LogisticRegression(C=1.0 )
clf.fit(xtrain_transform, y_train)
predictions = clf.predict_proba(xvalid_transform)

/Users/ddurgoji/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [27]:
from sklearn.metrics import roc_auc_score, accuracy

roc_auc_score(y_valid, predictions[:, 1])

ImportError: cannot import name 'accuracy' from 'sklearn.metrics' (/Users/ddurgoji/anaconda3/lib/python3.10/site-packages/sklearn/metrics/__init__.py)

In [26]:
clf.predict(xvalid_transform)

array([2, 2, 2, ..., 2, 1, 1])